In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Define the path to the UTKFace dataset
dataset_path = 'UTKFace'

# Create an ImageDataGenerator for data augmentation and loading
datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)
batch_size = 128

# Load and preprocess UTKFace dataset
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='input',  # Use 'input' as class_mode for autoencoder
    color_mode='grayscale'  # Use 'grayscale' for black and white images
)

In [4]:
# Define the Autoencoder model
input_img = layers.Input(shape=(256, 256, 1))  # Adjusted input shape

# Encoder
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
# Train the Autoencoder
autoencoder.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator), shuffle=True)

In [ ]:
# Display the original and reconstructed images
n = 10
for i in range(n):
    # Get a batch of images from the generator
    batch, _ = train_generator.next()

    # Encode and decode the batch
    decoded_batch = autoencoder.predict(batch)

    # Display the original and reconstructed images
    plt.figure(figsize=(20, 4))
    for j in range(n):
        # Original Images
        ax = plt.subplot(2, n, j + 1)
        plt.imshow(batch[j].reshape(256, 256), interpolation='nearest')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Reconstructed Images
        ax = plt.subplot(2, n, j + 1 + n)
        plt.imshow(decoded_batch[j].reshape(256, 256), interpolation='nearest')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()